# 1. Imports

In [1]:
import pandas as pd
import numpy as np

# 2. Função para carregar e filtrar os arquivos

In [10]:
def load_data(paths):
    lista_testes = []
    for i in range(10):
        # Abrindo o arquivo da temporada e convertendo em DataFrame:
        datasetTemporada = pd.read_csv(paths[i])
        df = pd.DataFrame(datasetTemporada)
        #Removendo coluna 'Time'(Tempo, em Inglês): (Atualizado em: 24_02_2021)
        lista_colunas = df.columns.values.tolist()
        if lista_colunas[2] == 'Time':
            df = df.drop(['Time'],axis=1)
        #Corrigindo diferença de quantidade de colunas: (Atualizado em: 24_02_2021)
        columns_size = df.shape[1]
        if columns_size < 105 and columns_size > 26:
            diff = 105 - columns_size
            j = 0 
            name_column = ''
            while j < diff:
                df[name_column] = ""
                name_column = name_column+"-"
                j = j + 1
        # Filtra o DataFrame original com apenas as colunas necessárias para análise;
        odds = np.arange(26,105) # atribuindo o intervalo das colunas 25 a 70, onde estão as odds não analisadas. (retorna valores espaçados igualmente dentro de um intervalo definido)
        df_filtrado = df.drop(df.columns[odds], axis = 1, inplace = True)  # Removendo as colunas de odds (permanentemente do DF original)
        df_filtrado = df.drop(columns = ['Div','Date','HTHG','HTAG','HTR','HF','AF','HC',
                                    'AC','HY','AY','HR','AR','Referee','B365D','B365A']) # Adicionei à remoção: B365D,B365A e as colunas dos times(Atualizado em: 25_02_2021)
        if len(df_filtrado) > 380:  # Corrigindo uma linha fantasma que surge no df, quando importa por google drive!
            df_filtrado = df_filtrado[:-1]
        # Adicionando a coluna oddLayH no DF "menor";
        oddsLayH = []
        for k in range(380):
            oddH = float(df_filtrado.iloc[k][9])
            oddLayH = 1/(1-(1/oddH))
            oddsLayH.append(np.round(oddLayH,2))
        df_filtrado['oddLayH'] = oddsLayH
        # Convertendo resultados de FTR para binario(Vitoria=1, Derrota=0 ,Empate=0) no DF "menor";
        listR = []
        for l in range(380): 
            if df_filtrado.iloc[l,4] == 'H':
                listR.append(1)  # Vitória do Mandante;
            else:
                listR.append(0)  # Derrota do Mandante ou Empate;
        df_filtrado['FTR'] = listR
        # Reposicionando as colunas:
        df_filtrado = df_filtrado[['HomeTeam','AwayTeam','FTHG','FTAG','HS','AS','HST','AST','FTR','B365H','oddLayH']]
        # Renomeando colunas B365H:
        df_filtrado.rename(columns = {'B365H':'oddH','HomeTeam':'Mandante','AwayTeam':'Visitante','FTHG':'GPM','FTAG':'GPV','HS':'CM','AS':'CV','HST':'CAAM','AST':'CAAV','FTR':'Resultado'}, inplace = True)
        lista_testes.append(df_filtrado)
    return lista_testes

## 2.1 Passando o endereço dos arquivos e carregando os arquivos

In [11]:
# Os endereços e nomes dos arquivos devem ser alterados de acordo com cada usuário;
paths = ["D:/Documentos_D/UFCG/Projeto_2020_2021/Tabelas Premier League/PremierLeague_2010_2011.csv",
         "D:/Documentos_D/UFCG/Projeto_2020_2021/Tabelas Premier League/PremierLeague_2011_2012.csv",
         "D:/Documentos_D/UFCG/Projeto_2020_2021/Tabelas Premier League/PremierLeague_2012_2013.csv",
         "D:/Documentos_D/UFCG/Projeto_2020_2021/Tabelas Premier League/PremierLeague_2013_2014.csv",
         "D:/Documentos_D/UFCG/Projeto_2020_2021/Tabelas Premier League/PremierLeague_2014_2015.csv",
         "D:/Documentos_D/UFCG/Projeto_2020_2021/Tabelas Premier League/PremierLeague_2015_2016.csv",
         "D:/Documentos_D/UFCG/Projeto_2020_2021/Tabelas Premier League/PremierLeague_2016_2017.csv",
         "D:/Documentos_D/UFCG/Projeto_2020_2021/Tabelas Premier League/PremierLeague_2017_2018.csv",
         "D:/Documentos_D/UFCG/Projeto_2020_2021/Tabelas Premier League/PremierLeague_2018_2019.csv",
         "D:/Documentos_D/UFCG/Projeto_2020_2021/Tabelas Premier League/PremierLeague_2019_2020.csv"]

In [ ]:
dfs = load_data(paths)

# 3. Funções para gerar as médias

In [44]:
def gera_df_medias_temporada(df_temporada, rodada_inicio):
    temp = df_temporada
    rodada = rodada_inicio
    novos_dfs_rodadas = []
    df_temp = pd.DataFrame()
    
    while rodada <= 38:
        #pula as X linhas iniciais,para gerar médias:
        rodada_teste = rodada
        max = (rodada_teste*10) - 10
        df = temp.iloc[0:max,0:8] # Gera um DataFrame com todas as linhas antes da rodada atual. A media sera calcula apenas com os valores anterior a rodada.
        # Coleta os times do campeonato:
        times = {}
        for i in range(40):
            time = df.iloc[i][0]
            if time in times:
                pass
            else:
                times[time] = []
        # Colentado as médias de cada time, para a rodada especificada:
        for nome_do_time in times.keys():
            '''Mandante'''
            temp_df = df.query('Mandante=="'+nome_do_time+'"')
            media_GPM = np.round(temp_df['GPM'].mean(),2)
            media_CM = np.round(temp_df['CM'].mean(),2)
            media_CAAM = np.round(temp_df['CAAM'].mean(),2)
            '''Visitante'''
            temp_df = df.query('Visitante=="'+nome_do_time+'"')
            media_GPV = np.round(temp_df['GPV'].mean(),2)
            media_CV = np.round(temp_df['CV'].mean(),2)
            media_CAAV = np.round(temp_df['CAAV'].mean(),2)

            times[nome_do_time].append(media_GPM)
            times[nome_do_time].append(media_GPV)
            times[nome_do_time].append(media_CM)
            times[nome_do_time].append(media_CV)
            times[nome_do_time].append(media_CAAM)
            times[nome_do_time].append(media_CAAV)
            
        # Gerando novo dataFrame com as médias da rodada específica:
        min = (rodada*10)-10
        max = min + 9
        df = temp
        df_copy = pd.DataFrame.copy(df,deep=True) 
        df_copy['GPM'] = df_copy['GPM'].astype(float)
        df_copy['GPV'] = df_copy['GPV'].astype(float)
        df_copy['CM'] = df_copy['CM'].astype(float)
        df_copy['CV'] = df_copy['CV'].astype(float)
        df_copy['CAAM'] = df_copy['CAAM'].astype(float)
        df_copy['CAAV'] = df_copy['CAAV'].astype(float)
        
        # Substituindo o dataframe original pelas medias de cada atributo:
        dic = times
        for time in times.keys():
            reqd_Index = df[df['Mandante']==time].index.tolist() # Encontra todos os índices onde o time é Mandante;
            for i in reqd_Index:
                if i >= min and i < min+10:
                    df_copy.iat[i,2] = np.round(dic[time][0],4)
                    df_copy.iat[i,4] = np.round(dic[time][2],4)
                    df_copy.iat[i,6] = np.round(dic[time][4],4)
            reqd_Index = df[df['Visitante']==time].index.tolist() # Encontra todos os índices onde o time é Visitante;
            for i in reqd_Index:
                if i >= min and i < min+10:
                    df_copy.iat[i,3] = np.round(dic[time][1],4)
                    df_copy.iat[i,5] = np.round(dic[time][3],4)
                    df_copy.iat[i,7] = np.round(dic[time][5],4)
        df_copy.rename(columns = {'GPM':'MediaGPM','GPV':'MediaGPV','CM':'MediaCM','CV':'MediaCV','CAAM':'MediaCAAM','CAAV':'MediaCAAV'}, inplace = True)
        df_medias_rodada_treino = df_copy[min:max+1]
        novos_dfs_rodadas.append(df_medias_rodada_treino)

        rodada += 1
    df_medias_temporada = df_temp.append(novos_dfs_rodadas,ignore_index=True)
    return df_medias_temporada

def gera_lista_medias_temporadas(dfs, rodada_inicio):  
    # Lista com os dfs das temporadas com as médias, rodada a rodada.
    dfs_medias_temporadas = []
    for df in dfs:
        temp_df = gera_df_medias_temporada(df, rodada_inicio)
        dfs_medias_temporadas.append(temp_df)
    return dfs_medias_temporadas

## 3.1 Gerando a litas com os dataframes com suas médias

In [49]:
lista_medias_temporadas = gera_lista_medias_temporadas(dfs, 6) 

In [48]:
for df in lista_medias_temporadas:
    display(df)

,Mandante,Visitante,MediaGPM,MediaGPV,MediaCM,MediaCV,MediaCAAM,MediaCAAV,Resultado,oddH,oddLayH
0,Aston Villa,Bolton,2.00,2.00,17.50,9.50,8.50,7.00,0,1.80,2.25
1,Blackburn,Fulham,1.00,1.00,11.00,12.00,4.50,7.00,0,2.00,2.00
2,Everton,Newcastle,2.00,0.50,18.50,9.00,10.00,4.50,0,1.62,2.61
3,Stoke,West Ham,1.50,0.00,17.50,9.50,11.00,2.00,0,1.91,2.10
4,Sunderland,Arsenal,1.50,1.50,9.00,14.00,3.00,8.00,0,5.50,1.22
...,...,...,...,...,...,...,...,...,...,...,...
335,Newcastle,West Brom,2.11,1.28,13.22,12.39,8.06,6.44,0,2.00,2.00
336,Stoke,Wigan,1.72,0.94,13.33,10.11,7.00,5.72,0,2.75,1.57
337,Tottenham,Birmingham,1.56,0.94,16.72,7.67,9.72,4.17,1,1.53,2.89
338,West Ham,Sunderland,1.33,0.94,13.39,10.78,7.06,5.00,0,2.00,2.00


,Mandante,Visitante,MediaGPM,MediaGPV,MediaCM,MediaCV,MediaCAAM,MediaCAAV,Resultado,oddH,oddLayH
0,Blackburn,Arsenal,0.50,1.00,15.00,14.00,7.00,8.50,1,4.20,1.31
1,Bolton,Norwich,1.00,1.00,13.00,11.00,8.50,4.50,0,1.80,2.25
2,Everton,Wigan,1.00,0.00,15.50,9.50,8.50,4.50,1,1.50,3.00
3,Swansea,West Brom,0.00,1.00,13.50,10.00,6.00,6.50,1,2.63,1.61
4,Wolves,QPR,1.00,0.50,13.50,11.50,6.50,6.00,0,1.91,2.10
...,...,...,...,...,...,...,...,...,...,...,...
335,Sunderland,Man United,1.44,2.00,12.17,11.39,6.72,7.33,0,7.00,1.17
336,Swansea,Liverpool,1.44,1.28,11.67,14.50,6.56,8.44,1,3.40,1.42
337,Tottenham,Fulham,2.06,0.67,18.72,9.83,11.17,5.67,1,1.40,3.50
338,West Brom,Arsenal,1.06,1.78,13.44,14.06,7.56,8.83,0,5.00,1.25


,Mandante,Visitante,MediaGPM,MediaGPV,MediaCM,MediaCV,MediaCAAM,MediaCAAV,Resultado,oddH,oddLayH
0,Southampton,Aston Villa,1.00,0.50,14.50,11.50,8.50,7.50,1,2.40,1.71
1,Swansea,Everton,2.50,1.50,12.00,15.50,8.50,8.00,0,3.00,1.50
2,West Brom,Reading,2.50,2.00,14.50,7.00,9.00,5.00,1,1.73,2.37
3,West Ham,Sunderland,2.00,1.00,12.50,3.50,8.00,2.50,0,2.10,1.91
4,Wigan,Fulham,1.00,1.00,10.50,14.00,4.50,12.00,0,2.40,1.71
...,...,...,...,...,...,...,...,...,...,...,...
335,Swansea,Fulham,1.56,1.06,14.33,8.89,8.28,5.33,0,1.80,2.25
336,Tottenham,Sunderland,1.56,1.17,17.94,8.17,11.06,5.00,1,1.29,4.45
337,West Brom,Man United,1.50,2.00,14.00,12.22,7.44,7.11,0,4.50,1.29
338,West Ham,Reading,1.67,1.00,12.56,9.61,7.56,4.83,1,1.67,2.49


,Mandante,Visitante,MediaGPM,MediaGPV,MediaCM,MediaCV,MediaCAAM,MediaCAAV,Resultado,oddH,oddLayH
0,Chelsea,Fulham,2.00,0.50,18.50,4.00,4.00,1.50,1,1.30,4.33
1,Liverpool,Southampton,1.00,0.50,18.00,9.00,8.00,2.00,0,1.60,2.67
2,Newcastle,Hull,0.50,0.00,20.00,7.00,4.50,2.00,0,1.85,2.18
3,Norwich,Aston Villa,1.50,2.00,11.50,8.00,3.00,3.50,0,2.40,1.71
4,West Brom,Sunderland,0.00,1.00,10.50,9.00,1.00,2.50,1,2.05,1.95
...,...,...,...,...,...,...,...,...,...,...,...
335,Norwich,Arsenal,0.94,1.67,14.11,12.50,4.00,4.89,0,4.50,1.29
336,Southampton,Man United,1.72,1.89,15.78,12.00,5.33,4.28,0,2.63,1.61
337,Sunderland,Swansea,1.11,1.00,15.00,13.17,3.89,3.56,0,2.30,1.77
338,Tottenham,Aston Villa,1.50,0.94,15.83,10.72,5.11,3.22,1,1.50,3.00


,Mandante,Visitante,MediaGPM,MediaGPV,MediaCM,MediaCV,MediaCAAM,MediaCAAV,Resultado,oddH,oddLayH
0,Aston Villa,Arsenal,1.00,1.50,10.00,18.50,2.00,4.50,0,4.75,1.27
1,Burnley,Sunderland,0.50,1.00,9.00,11.50,2.50,4.00,0,2.50,1.67
2,Newcastle,Hull,1.50,1.00,15.00,10.00,2.00,3.50,0,2.20,1.83
3,QPR,Stoke,0.50,1.00,18.50,13.00,5.50,2.50,0,2.80,1.56
4,Swansea,Southampton,2.00,2.00,12.50,15.00,6.50,7.00,0,2.50,1.67
...,...,...,...,...,...,...,...,...,...,...,...
335,Hull,Man United,1.06,1.17,12.61,12.22,4.00,4.11,0,3.20,1.45
336,Leicester,QPR,1.28,1.00,14.06,11.50,4.22,3.28,1,1.60,2.67
337,Man City,Southampton,2.33,0.94,19.22,11.39,6.83,3.78,1,1.57,2.75
338,Newcastle,West Ham,1.33,1.06,13.94,11.39,4.28,3.78,1,1.83,2.20


,Mandante,Visitante,MediaGPM,MediaGPV,MediaCM,MediaCV,MediaCAAM,MediaCAAV,Resultado,oddH,oddLayH
0,Arsenal,Stoke,0.00,1.50,20.50,11.00,5.50,4.00,1,1.29,4.45
1,Crystal Palace,Man City,1.50,2.50,13.50,17.50,5.00,8.00,0,5.50,1.22
2,Everton,Chelsea,1.00,1.50,10.00,12.50,3.00,4.00,1,4.00,1.33
3,Man United,Liverpool,0.50,0.50,14.50,11.50,4.50,5.50,1,2.00,2.00
4,Norwich,Bournemouth,1.00,2.00,19.00,14.00,6.50,4.50,1,2.40,1.71
...,...,...,...,...,...,...,...,...,...,...,...
335,Stoke,West Ham,1.11,1.67,11.78,12.44,3.33,4.00,1,3.50,1.40
336,Swansea,Man City,1.06,1.28,12.44,13.72,3.39,5.00,0,6.00,1.20
337,Watford,Sunderland,1.00,1.28,12.72,10.28,3.78,3.72,0,2.05,1.95
338,West Brom,Liverpool,1.06,1.61,11.33,14.06,3.33,5.11,0,2.40,1.71


,Mandante,Visitante,MediaGPM,MediaGPV,MediaCM,MediaCV,MediaCAAM,MediaCAAV,Resultado,oddH,oddLayH
0,Chelsea,Liverpool,2.50,1.67,19.00,18.33,8.00,5.00,0,2.25,1.80
1,Everton,Middlesbrough,1.00,1.00,16.50,7.50,7.50,2.50,1,1.67,2.49
2,Hull,Arsenal,1.00,1.50,11.00,11.50,3.50,5.50,0,6.00,1.20
3,Leicester,Burnley,1.00,0.00,10.50,6.00,3.50,0.00,1,1.50,3.00
4,Man City,Bournemouth,2.50,0.50,19.00,9.00,4.50,4.00,1,1.29,4.45
...,...,...,...,...,...,...,...,...,...,...,...
335,Liverpool,Middlesbrough,2.33,0.56,17.00,7.50,6.50,2.11,1,1.14,8.14
336,Man United,Crystal Palace,1.33,1.44,18.22,11.56,6.44,3.50,1,2.50,1.67
337,Southampton,Stoke,0.94,0.89,15.83,10.17,4.78,3.06,0,1.80,2.25
338,Swansea,West Brom,1.39,0.83,11.72,9.11,4.50,2.56,1,2.10,1.91


,Mandante,Visitante,MediaGPM,MediaGPV,MediaCM,MediaCV,MediaCAAM,MediaCAAV,Resultado,oddH,oddLayH
0,Bournemouth,Brighton,0.50,0.00,7.50,10.50,2.50,2.00,1,2.15,1.87
1,Crystal Palace,Southampton,0.00,0.00,15.00,6.00,3.50,3.00,0,3.00,1.50
2,Huddersfield,Leicester,0.50,1.50,11.50,8.50,4.50,3.50,0,3.50,1.40
3,Liverpool,Burnley,2.50,2.00,20.50,11.50,11.50,4.00,0,1.29,4.45
4,Newcastle,Stoke,1.50,0.50,11.00,12.50,5.50,1.50,1,2.38,1.72
...,...,...,...,...,...,...,...,...,...,...,...
335,Newcastle,Chelsea,1.00,1.78,11.56,13.56,3.61,4.78,1,6.50,1.18
336,Southampton,Man City,1.11,2.44,13.56,16.89,4.11,6.00,0,9.00,1.12
337,Swansea,Stoke,0.89,0.72,10.44,9.50,3.22,3.39,0,1.90,2.11
338,Tottenham,Leicester,1.94,1.50,18.83,9.94,6.33,3.56,1,1.33,4.03


,Mandante,Visitante,MediaGPM,MediaGPV,MediaCM,MediaCV,MediaCAAM,MediaCAAV,Resultado,oddH,oddLayH
0,Bournemouth,Leicester,2.00,1.50,14.50,10.50,4.50,4.50,1,2.45,1.69
1,Chelsea,Cardiff,2.50,0.00,24.00,12.00,8.50,2.50,1,1.18,6.56
2,Huddersfield,Crystal Palace,0.00,1.50,5.50,9.50,1.00,6.00,0,3.30,1.43
3,Man City,Fulham,4.00,1.50,28.00,10.00,11.00,4.00,1,1.12,9.33
4,Newcastle,Arsenal,1.00,2.50,10.50,16.00,2.00,8.50,0,4.20,1.31
...,...,...,...,...,...,...,...,...,...,...,...
335,Liverpool,Wolves,2.94,1.06,17.89,11.11,6.72,3.56,1,1.30,4.33
336,Man United,Cardiff,1.83,0.61,14.33,9.67,6.44,3.00,0,1.28,4.57
337,Southampton,Huddersfield,1.44,0.61,13.78,10.39,4.78,3.22,0,1.44,3.27
338,Tottenham,Everton,1.78,1.22,16.50,11.06,5.67,4.17,0,2.20,1.83


,Mandante,Visitante,MediaGPM,MediaGPV,MediaCM,MediaCV,MediaCAAM,MediaCAAV,Resultado,oddH,oddLayH
0,Liverpool,Newcastle,3.50,1.00,20.00,9.00,6.00,3.00,1,1.14,8.14
1,Brighton,Burnley,0.50,1.00,14.00,15.50,3.50,4.50,0,2.15,1.87
2,Man United,Leicester,2.50,1.50,16.50,11.00,4.00,2.50,1,1.95,2.05
3,Sheffield United,Southampton,1.00,1.00,11.50,11.50,3.00,3.50,0,2.45,1.69
4,Tottenham,Crystal Palace,1.50,1.00,24.00,5.50,4.50,3.50,1,1.36,3.78
...,...,...,...,...,...,...,...,...,...,...,...
335,Leicester,Man United,1.94,1.33,13.78,13.17,4.72,5.28,0,3.25,1.44
336,Man City,Norwich,2.89,0.39,20.17,10.33,6.94,3.00,1,1.08,13.50
337,Newcastle,Liverpool,1.06,1.67,12.78,14.56,4.17,5.33,0,7.50,1.15
338,Southampton,Sheffield United,1.00,0.78,12.61,7.78,4.06,2.33,1,2.15,1.87
